In [1]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pandas.io.json import json_normalize

In [2]:
## Connect to our service account
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./Jupyter-and-Google-Sheet-584f47a17e35.json', scope)
gc = gspread.authorize(credentials)

In [3]:
##Get candidate data sheet from Google Drive
spreadsheet_key = '1f0OwtmuTk1fTdhnn4tuvVcPCZjdb00D79dWw4RFhYs0'
book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet("Candidate Data")
table = worksheet.get_all_values()
table[:8]

[['Order',
  'Candidate',
  'Position',
  'Start Date',
  'End Date',
  'Years of Experience',
  'Points',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['45',
  'Donald Trump',
  'President',
  '1/20/2017',
  '1/20/2021',
  '4.002739726',
  '104',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['45',
  'Donald Trump',
  'Secretary of State',
  '',
  '',
  '0',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['45',
  'Donald Trump',
  'Secretary of Defense',
  '',
  '',
  '0',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['45',
  'Donald Trump',
  'Secretary of Treasury',
  '',
  '',
  '0',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['45',
  'Donald Trump',
  'Vice President',
  '',
  '',
  '0',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['45',
  'Donald Trump',
  'Attorney General',
  '',

In [4]:
##Convert table data into a dataframe
df = pd.DataFrame(table[1:], columns=table[0])

##Only keep columns we need
df = df[['Order', 'Candidate', 'Position', 'Start Date', 'End Date', 'Years of Experience', 'Points']]

##Convert number strings to floats and ints
df = df.apply(pd.to_numeric, errors='ignore')

##Convert date strings to datetime format
df['End Date'] = pd.to_datetime(df['End Date'],infer_datetime_format=True)
df['Start Date'] = pd.to_datetime(df['Start Date'],infer_datetime_format=True)

df.head()

,Order,Candidate,Position,Start Date,End Date,Years of Experience,Points
0,45,Donald Trump,President,2017-01-20,2021-01-20,4.00274,104.0
1,45,Donald Trump,Secretary of State,NaT,NaT,0.00000,0.0
2,45,Donald Trump,Secretary of Defense,NaT,NaT,0.00000,0.0
3,45,Donald Trump,Secretary of Treasury,NaT,NaT,0.00000,0.0
4,45,Donald Trump,Vice President,NaT,NaT,0.00000,0.0


In [5]:
## Do some manipulation of the data
candidate_groups = df.groupby('Candidate')
scores_df = candidate_groups.sum()
scores_df['Order'] = candidate_groups.first()
scores_df.head()

,Order,Years of Experience,Points
Candidate,,,
Abraham Lincoln,16,6.336986,114.00
Andrew Jackson,7,19.884932,246.85
Andrew Johnson,17,25.443836,204.45
Barack Obama,44,12.054795,208.55
Benjamin Harrison,23,12.887671,137.70


In [6]:
## Save the data back to a new sheet in the dataframe
from df2gspread import df2gspread as d2g

wks_name = 'Jupyter Manipulated Data'

d2g.upload(scores_df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Jupyter Manipulated Data' id:o74wj08>